In [ ]:
################# Import Statements #####################

# !apt-get update
# !apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin

import csv
import pandas as pd
import requests
from random import randint
from time import sleep
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.keys import Keys 
from scrapy.selector import Selector
from selenium.webdriver.common.by import By
from defaultlist import defaultlist
from parsel import Selector
from collections import OrderedDict
import random
from selenium.webdriver.support.ui import WebDriverWait 

from http_request_randomizer.requests.proxy.requestProxy import RequestProxy
req_proxy = RequestProxy() #you may get different number of proxy when  you run this at each time
proxies = req_proxy.get_proxy_list() #this will create proxy list


################# Get the proxy(ies) that resolve LinkedIn in less than 5 seconds #####################
def get_proxy(proxies=proxies):
  #Get only the proxies with the ports 8080
  port_80 = []
  for i in proxies:
      if '8080' in i.get_address():
          port_80.append(i.get_address())

  PROXY = []

  #Get some ports that will resolve LinkedIn in less than 5 seconds
  for i in port_80[randint(1,len(proxies))]:
      proxy = {"http": i, "https": i}  
      try:
          resp = requests.get('https://www.linkedin.com', proxies=proxy, timeout=5)
          PROXY.append(i)
      except:
          pass
  return PROXY


################ Using Webdriver to call LinkedIn #####################

def linkedin_login(driver, email, password, PROXY_relevant):
    """Logs you into LinkedIn in the ChromeWebdriver."""
    webdriver.DesiredCapabilities.CHROME['proxy'] = {
        "httpProxy": PROXY_relevant,
        "ftpProxy": PROXY_relevant,
        "sslProxy": PROXY_relevant,
        "proxyType": "MANUAL"
    }

    url = 'https://www.linkedin.com'

    driver.get(url)
    username = driver.find_element_by_class_name('input__input')
    sleep(3)
    username.send_keys(email)
    passkey = driver.find_element_by_id('session_password')
    sleep(2)
    passkey.send_keys(password)
    log_in_button = driver.find_element_by_xpath('//*[@type="submit"]')
    log_in_button.click()


email = '<enter your email-id>'
password = '<enter your password>'

# Download Chrome driver from here  https://chromedriver.chromium.org/downloads and then provide the location where the webdriver is being stored.
driver = webdriver.Chrome('/usr/bin/chromedriver')

PROXY_relevant = get_proxy()[-1]

linkedin_login(driver, email, password, PROXY_relevant)
# This block will simply open and log you into LinkedIn.
# Once done, you have to head over to Google to search for the profiles for which you want the data.

################ Get LinkedIn profiles from Google #####################

sleep(5)
#In a new page, in Selenium webdriver open Google.
driver.execute_script("window.open('https://www.google.com/','_self')")

sleep(5)
# locate search form by_name
search_query = driver.find_element_by_name('q')

if driver.find_element_by_name('q').get_attribute('value'):
    driver.find_element_by_name('q').clear()

search_term = input('Please enter search term: ')
location = input('Please enter location: ')

#Clear if the search box already contains any string
# search_query.send_keys(Keys.CLEAR)

sleep(3)

if location == "" or " " or None:
    # send_keys() to simulate the search text key strokes
    search_query.send_keys(f'site:linkedin.com/in/ AND {search_term}')
else:
    search_query.send_keys(f'site:linkedin.com/in/ AND {search_term} AND {location}')
                           
# .send_keys() to simulate the return/enter key 
search_query.send_keys(Keys.RETURN)

################ Get profile data from their respective pages #####################
def extract_all_profile_data(driver, linkedin_urls):
    """Extract data from each page using xpaths."""
    # Create a dictionary of lists to store the scarped data.
    data = {}
    url = defaultlist(lambda: 'empty')
    name = defaultlist(lambda: 'empty')
    job_title = defaultlist(lambda: 'empty')
    company = defaultlist(lambda: 'empty')
    education = defaultlist(lambda: 'empty')
    designation = defaultlist(lambda: 'empty')
    location = defaultlist(lambda: 'empty')
    qualification = defaultlist(lambda: 'empty')

    # Using Selectors to scrape data. 
    # Each loop iteration goes through one page and extracts the data in the specified locations (location specifoied by the xpaths).
    for page_num, page_url in enumerate(linkedin_urls):

        if page_num == randint(0, len(linkedin_urls)):
            sleep(randint(10, 30))
        sleep(randint(1, 5))
        url.append(page_url)
        driver.get(page_url)
        text = driver.page_source

        sleep(randint(5, 20))
        try:
            name.append(Selector(text=text).xpath(
                '//*[@class="inline t-24 t-black t-normal break-words"]/text()').get().strip())
        except:
            name.append('No info')

        try:
            job_title.append(
                Selector(text=text).xpath('//*[@class="mt1 t-18 t-black t-normal break-words"]/text()').get().strip())
        except:
            job_title.append('No info')

        try:
            if Selector(text=text).xpath(
                    '//*[@class="text-align-left ml2 t-14 t-black t-bold full-width lt-line-clamp lt-line-clamp--multi-line ember-view"]/text()').get().strip() is None:
                company.append(Selector(text=text).xpath(
                    '//section/div[1]/section/ul/li[1]/section//div[2]/h3[@class="t-16 t-black t-bold"]/span[2]/text()').get().strip())
            else:
                company.append(Selector(text=text).xpath(
                    '//*[@class="text-align-left ml2 t-14 t-black t-bold full-width lt-line-clamp lt-line-clamp--multi-line ember-view"]/text()').get().strip())
        #         company.append(Selector(text=text).xpath('//*[@class="text-align-left ml2 t-14 t-black t-bold full-width lt-line-clamp lt-line-clamp--multi-line ember-view"]/text()').get().strip())
        except:
            company.append('No info')

        try:
            if Selector(text=text).xpath(
                    '//section/div[2]/div[2]/div[1]/h2[@class="mt1 t-18 t-black t-normal break-words"]/text()').get().strip() is None:
                designation.append(Selector(text=text).xpath(
                    '//section/div[1]/section/ul/li[1]/section/ul/li[1]//h3[@class="t-14 t-black t-bold"]/span[2]/text()').get().strip())
            else:
                designation.append(Selector(text=text).xpath(
                    '//section/div[2]/div[2]/div[1]/h2[@class="mt1 t-18 t-black t-normal break-words"]/text()').get().strip())
        except:
            designation.append('No info')

        sleep(randint(2, 5))
        try:
            if Selector(text=text).xpath(
                    '//section/div[2]/div[2]/div[1]//li[@class="t-16 t-black t-normal inline-block"]/text()').get().strip() is None:
                location.append(Selector(text=text).xpath(
                    '//section/div[1]/section/ul/li[1]/section/ul/li[1]//h3[@class="t-14 t-black t-bold"]/span[2]/text()').get().strip())
            else:
                location.append(Selector(text=text).xpath(
                    '//section/div[2]/div[2]/div[1]//li[@class="t-16 t-black t-normal inline-block"]/text()').get().strip())
        except:
            location.append('No info')

        try:
            #             if Selector(text=text).xpath('//section/div[2]/section/ul/li[1]//a/div[2]/div/h3[@class="pv-entity__school-name t-16 t-black t-bold"]/text()').get().strip() == None or '':
            education.append(Selector(text=text).xpath(
                '//h3[@class="pv-entity__school-name t-16 t-black t-bold"]/text()').get().strip())
        #             else:
        #             education.append(Selector(text=text).xpath('//section/div[2]/section/ul/li[1]//a/div[2]/div/h3[@class="pv-entity__school-name t-16 t-black t-bold"]/text()').get().strip())
        except:
            education.append('No info')

        try:
            #             if Selector(text=text).xpath('//section/div[2]/section/ul/li[1]//a/div[2]/div/p[2]/span[@class="pv-entity__comma-item"]/text()').get().strip() == None or '':
            qualification.append(
                Selector(text=text).xpath('//div//span[2][@class="pv-entity__comma-item"]/text()').get().strip())
        #             else:
        #             qualification.append(Selector(text=text).xpath('//section/div[2]/section/ul/li[1]//a/div[2]/div/p[2]/span[@class="pv-entity__comma-item"]/text()').get().strip())  
        except:
            qualification.append('No info')

        sleep(randint(2, 4))

    data['URL'] = url
    data['Name'] = name
    data['Job title'] = job_title
    data['Company'] = company
    data['Designation'] = designation
    data['Location'] = location
    data['Education'] = education
    data['Qualification'] = qualification

    #Data contains the ultimate data from all pages.
    return data


################ Save the data into CSV files #####################

Users = pd.DataFrame.from_dict(extract_all_profile_data(driver,linkedin_urls))
path = '/home/usr/Desktop/CSV/' #Change this path according to your file structure.
Users.to_csv(path_or_buf = path + 'linkedin_profile_data.csv',index=False)